In [ ]:
# Section 1: Load Dataset and Show Samples
import sys
import os
sys.path.append('../sql_evaluation_library/src')

from sql_eval_lib.dataset.models import DatasetItem
from sql_eval_lib.dataset.loaders.huggingface_loader import HuggingFaceLoader
import pandas as pd

print("🔍 SECTION 1: Dataset Loading & Sample Exploration")
print("=" * 50)

loader = HuggingFaceLoader()

dataset = loader.load('gretelai/synthetic_text_to_sql', split='train')
print(f"✅ Dataset loaded successfully with {len(dataset.items)} items")


🔍 SECTION 1: Dataset Loading & Sample Exploration
✅ Dataset loaded successfully with 100000 items


In [11]:
dataset.items[0].input

{'id': 5097,
 'domain': 'forestry',
 'domain_description': 'Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.',
 'sql_complexity': 'single join',
 'sql_complexity_description': 'only one join (specify inner, outer, cross)',
 'sql_task_type': 'analytics and reporting',
 'sql_task_type_description': 'generating reports, dashboards, and analytical insights',
 'sql_prompt': 'What is the total volume of timber sold by each salesperson, sorted by salesperson?',
 'sql_context': "CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');",
 'sql'

In [ ]:

# Convert to the format we need for evaluation
sample_data = []

for i, item in enumerate(dataset.items[:2]):  # Take first 2 samples
    # The HuggingFaceLoader puts the raw data in the input field
    raw_data = item.input
    
    # Convert to our evaluation format
    converted_item = DatasetItem(
        input={
            "question": raw_data.get('sql_prompt', ''),
            "database_schema": raw_data.get('sql_context', ''),  # Schema info is in sql_context
            "domain": raw_data.get('domain', ''),
            "domain_description": raw_data.get('domain_description', '')
        },
        expected_output={
            "sql": raw_data.get('sql', '')
        },
        metadata={
            "difficulty": raw_data.get('sql_complexity', 'unknown'),
            "category": raw_data.get('sql_task_type', 'unknown'),
            "complexity_description": raw_data.get('sql_complexity_description', ''),
            "task_description": raw_data.get('sql_task_type_description', ''),
            "sql_explanation": raw_data.get('sql_explanation', ''),
            "original_id": raw_data.get('id', f'sample_{i+1}')
        }
    )
    sample_data.append(converted_item)
    


print(f"📊 Loaded {len(sample_data)} sample SQL evaluation items")
print("\n🔍 Sample Dataset Items:")
for i, item in enumerate(sample_data, 1):
    print(f"\n--- Sample {i} ---")
    print(f"Question: {item.input.get('question', 'N/A')}")
    print(f"SQL: {item.expected_output.get('sql', 'N/A')}")
    print(f"Difficulty: {item.metadata.get('difficulty', 'N/A')}")
    print(f"Category: {item.metadata.get('category', 'N/A')}")
    
    # Show domain info if available
    domain = item.input.get('domain')
    if domain:
        print(f"Domain: {domain}")
    
    # Show schema (truncated if too long)
    schema = item.input.get('database_schema', 'N/A')
    if len(str(schema)) > 200:
        print(f"Schema: {str(schema)[:200]}...")
    else:
        print(f"Schema: {schema}")
    
    # Show explanation if available
    explanation = item.metadata.get('sql_explanation')
    if explanation:
        print(f"Explanation: {explanation[:150]}...")

print(f"\n✅ Section 1 Complete: Dataset loaded with {len(sample_data)} samples")


In [ ]:
# Section 2: Run Evaluation with 5 Samples (Without Langfuse)
from sql_eval_lib.evaluation import SQLLLMEvaluator, SQLExecutionEvaluator, EvaluationSuite, SQLEvaluationContext
from sql_eval_lib.evaluation.metrics import SQLSyntaxMetric, SQLExecutionAccuracyMetric, SemanticSimilarityMetric

print("\n🚀 SECTION 2: Standalone SQL Evaluation")
print("=" * 50)

# Note: Add your OpenAI API key here
OPENAI_API_KEY = "your-openai-api-key-here"  # Replace with your actual key

# Initialize evaluation context
eval_context = SQLEvaluationContext(
    database_type="sqlite",  # Using SQLite for demonstration
    timeout_seconds=30
)

# Initialize evaluators
print("🔧 Initializing evaluators...")

try:
    # LLM Evaluator for semantic and syntax analysis
    llm_evaluator = SQLLLMEvaluator(
        api_key=OPENAI_API_KEY,
        model="gpt-3.5-turbo",
        max_tokens=1000
    )
    print("✅ LLM Evaluator initialized")
except Exception as e:
    print(f"⚠️ LLM Evaluator initialization failed: {e}")
    llm_evaluator = None

# Execution Evaluator for SQL execution testing
execution_evaluator = SQLExecutionEvaluator()
print("✅ Execution Evaluator initialized")

# Initialize metrics
metrics = [
    SQLSyntaxMetric(),
    SQLExecutionAccuracyMetric(),
    SemanticSimilarityMetric()
]
print(f"✅ {len(metrics)} metrics initialized")

# Create evaluation suite
evaluation_suite = EvaluationSuite(
    evaluators=[evaluator for evaluator in [llm_evaluator, execution_evaluator] if evaluator is not None],
    metrics=metrics
)

# Run evaluation on first 5 samples
print(f"\n🧪 Running evaluation on {min(5, len(sample_data))} samples...")
evaluation_results = []

for i, item in enumerate(sample_data[:5], 1):
    print(f"\n--- Evaluating Sample {i} ---")
    print(f"Question: {item.input.get('question', 'N/A')}")
    
    try:
        # Run evaluation
        result = evaluation_suite.evaluate_item(item, eval_context)
        evaluation_results.append({
            'sample_id': i,
            'question': item.input.get('question', 'N/A'),
            'result': result,
            'status': 'success'
        })
        print(f"✅ Sample {i} evaluated successfully")
        
        # Display basic results
        if hasattr(result, 'metrics'):
            for metric_name, metric_value in result.metrics.items():
                print(f"  {metric_name}: {metric_value}")
                
    except Exception as e:
        print(f"❌ Sample {i} evaluation failed: {e}")
        evaluation_results.append({
            'sample_id': i,
            'question': item.input.get('question', 'N/A'),
            'error': str(e),
            'status': 'failed'
        })

# Summary
successful_evaluations = len([r for r in evaluation_results if r['status'] == 'success'])
print(f"\n📊 Evaluation Summary:")
print(f"  Total samples: {len(evaluation_results)}")
print(f"  Successful: {successful_evaluations}")
print(f"  Failed: {len(evaluation_results) - successful_evaluations}")

print(f"\n✅ Section 2 Complete: Standalone evaluation finished")


In [ ]:
# Section 3: Ensure Langfuse Self-Hosted is Ready
import subprocess
import time
import requests
from sql_eval_lib.langfuse import LangfuseDeployment

print("\n🐳 SECTION 3: Langfuse Self-Hosted Setup Verification")
print("=" * 50)

# Check if Docker is available
def check_docker():
    try:
        result = subprocess.run(['docker', '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ Docker is available: {result.stdout.strip()}")
            return True
        else:
            print("❌ Docker is not available")
            return False
    except FileNotFoundError:
        print("❌ Docker command not found")
        return False

# Check if Docker Compose is available
def check_docker_compose():
    try:
        result = subprocess.run(['docker-compose', '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ Docker Compose is available: {result.stdout.strip()}")
            return True
        else:
            # Try docker compose (newer syntax)
            result = subprocess.run(['docker', 'compose', 'version'], capture_output=True, text=True)
            if result.returncode == 0:
                print(f"✅ Docker Compose is available: {result.stdout.strip()}")
                return True
            print("❌ Docker Compose is not available")
            return False
    except FileNotFoundError:
        print("❌ Docker Compose command not found")
        return False

# Check for existing docker-compose.yml
def check_docker_compose_file():
    docker_compose_path = "../docker-compose.yml"
    if os.path.exists(docker_compose_path):
        print(f"✅ Found docker-compose.yml at {docker_compose_path}")
        return True
    else:
        print(f"❌ No docker-compose.yml found at {docker_compose_path}")
        return False

print("🔍 Checking Docker environment...")
docker_available = check_docker()
docker_compose_available = check_docker_compose()
docker_compose_file_exists = check_docker_compose_file()

if all([docker_available, docker_compose_available, docker_compose_file_exists]):
    print("\n🚀 Attempting to deploy Langfuse...")
    
    try:
        # Initialize Langfuse deployment
        deployment = LangfuseDeployment(
            compose_file_path="../docker-compose.yml",
            project_name="sql_eval_langfuse"
        )
        
        print("📦 Starting Langfuse deployment...")
        deployment.deploy()
        
        print("⏳ Waiting for Langfuse to become healthy...")
        if deployment.wait_for_healthy(timeout=120):
            print("✅ Langfuse is running and healthy!")
            
            # Test connection
            try:
                response = requests.get("http://localhost:3000/api/public/health", timeout=10)
                if response.status_code == 200:
                    print("✅ Langfuse API is responding")
                else:
                    print(f"⚠️ Langfuse API returned status {response.status_code}")
            except Exception as e:
                print(f"⚠️ Could not test Langfuse API: {e}")
                
        else:
            print("❌ Langfuse failed to become healthy within timeout")
            
    except Exception as e:
        print(f"❌ Langfuse deployment failed: {e}")
        print("💡 You may need to:")
        print("   - Check Docker daemon is running")
        print("   - Ensure ports 3000 and 5432 are available")
        print("   - Review docker-compose.yml configuration")
        
else:
    print("\n❌ Prerequisites not met for Langfuse deployment")
    print("💡 Required:")
    print("   - Docker installed and running")
    print("   - Docker Compose available")
    print("   - docker-compose.yml file present")

print(f"\n✅ Section 3 Complete: Langfuse setup verification finished")


In [ ]:
# Section 4: Create Dataset with 5 Samples Through Langfuse
from sql_eval_lib.langfuse import LangfuseClient
from sql_eval_lib.dataset.langfuse_integration import LangfuseDatasetManager

print("\n🔗 SECTION 4: Langfuse-Integrated Evaluation")
print("=" * 50)

# Langfuse configuration
LANGFUSE_PUBLIC_KEY = "pk-lf-1234567890abcdef"  # Replace with actual key
LANGFUSE_SECRET_KEY = "sk-lf-1234567890abcdef"  # Replace with actual key
LANGFUSE_HOST = "http://localhost:3000"

try:
    print("🔧 Initializing Langfuse client...")
    
    # Initialize Langfuse client
    langfuse_client = LangfuseClient(
        public_key=LANGFUSE_PUBLIC_KEY,
        secret_key=LANGFUSE_SECRET_KEY,
        host=LANGFUSE_HOST
    )
    
    # Test connection
    print("🔍 Testing Langfuse connection...")
    if langfuse_client.test_connection():
        print("✅ Langfuse connection successful!")
        
        # Initialize dataset manager
        dataset_manager = LangfuseDatasetManager(langfuse_client)
        
        # Create a new dataset
        dataset_name = f"sql_evaluation_samples_{int(time.time())}"
        print(f"\n📊 Creating dataset: {dataset_name}")
        
        try:
            # Create dataset in Langfuse
            dataset_id = dataset_manager.create_dataset(
                name=dataset_name,
                description="SQL evaluation samples for interactive notebook testing"
            )
            print(f"✅ Dataset created with ID: {dataset_id}")
            
            # Upload sample data to Langfuse
            print(f"\n📤 Uploading {len(sample_data)} samples to Langfuse...")
            
            uploaded_count = 0
            for i, item in enumerate(sample_data, 1):
                try:
                    # DatasetItem is already in the correct format for Langfuse
                    langfuse_item = {
                        "input": item.input,
                        "expected_output": item.expected_output,
                        "metadata": item.metadata
                    }
                    
                    # Upload to Langfuse
                    item_id = dataset_manager.add_item(
                        dataset_id=dataset_id,
                        item=langfuse_item
                    )
                    
                    print(f"✅ Sample {i} uploaded (ID: {item_id})")
                    uploaded_count += 1
                    
                except Exception as e:
                    print(f"❌ Failed to upload sample {i}: {e}")
            
            print(f"\n📊 Upload Summary:")
            print(f"  Total samples: {len(sample_data)}")
            print(f"  Successfully uploaded: {uploaded_count}")
            print(f"  Failed: {len(sample_data) - uploaded_count}")
            
            # Retrieve and verify dataset
            print(f"\n🔍 Verifying dataset in Langfuse...")
            try:
                retrieved_items = dataset_manager.get_dataset_items(dataset_id)
                print(f"✅ Verified: {len(retrieved_items)} items in dataset")
                
                # Show first item as verification
                if retrieved_items:
                    first_item = retrieved_items[0]
                    print(f"\n🔍 First item verification:")
                    print(f"  Question: {first_item.get('input', {}).get('question', 'N/A')}")
                    print(f"  Expected SQL: {first_item.get('expected_output', 'N/A')[:100]}...")
                    
            except Exception as e:
                print(f"⚠️ Could not verify dataset: {e}")
            
            # Run evaluation with Langfuse tracking
            print(f"\n🧪 Running Langfuse-tracked evaluation...")
            
            langfuse_evaluation_results = []
            for i, item in enumerate(sample_data[:3], 1):  # Test with 3 samples
                print(f"\n--- Langfuse Evaluation Sample {i} ---")
                
                try:
                    # Create a trace in Langfuse for this evaluation
                    trace = langfuse_client.trace(
                        name=f"sql_evaluation_sample_{i}",
                        metadata={"sample_id": i, "dataset": dataset_name}
                    )
                    
                    # Run evaluation (reuse previous evaluation logic)
                    result = evaluation_suite.evaluate_item(item, eval_context)
                    
                    # Log results to Langfuse
                    trace.generation(
                        name="sql_evaluation",
                        input={"question": item.input.get('question'), "sql": item.expected_output.get('sql')},
                        output={"evaluation_result": str(result)},
                        metadata=item.metadata
                    )
                    
                    langfuse_evaluation_results.append({
                        'sample_id': i,
                        'trace_id': trace.id,
                        'status': 'success'
                    })
                    
                    print(f"✅ Sample {i} evaluated and logged to Langfuse")
                    
                except Exception as e:
                    print(f"❌ Sample {i} Langfuse evaluation failed: {e}")
                    langfuse_evaluation_results.append({
                        'sample_id': i,
                        'error': str(e),
                        'status': 'failed'
                    })
            
            # Final summary
            successful_langfuse = len([r for r in langfuse_evaluation_results if r['status'] == 'success'])
            print(f"\n📊 Langfuse Evaluation Summary:")
            print(f"  Dataset created: {dataset_name}")
            print(f"  Items uploaded: {uploaded_count}/{len(sample_data)}")
            print(f"  Evaluations tracked: {successful_langfuse}/{len(langfuse_evaluation_results)}")
            
        except Exception as e:
            print(f"❌ Dataset creation/upload failed: {e}")
            
    else:
        print("❌ Langfuse connection failed")
        print("💡 Please check:")
        print("   - Langfuse is running (Section 3)")
        print("   - API keys are correct")
        print("   - Host URL is accessible")
        
except Exception as e:
    print(f"❌ Langfuse client initialization failed: {e}")
    print("💡 Please ensure:")
    print("   - Langfuse is deployed and running")
    print("   - API credentials are configured")
    print("   - Network connectivity to Langfuse host")

print(f"\n✅ Section 4 Complete: Langfuse-integrated evaluation finished")

print(f"\n🎉 ALL SECTIONS COMPLETE!")
print("=" * 50)
print("📋 Summary of completed sections:")
print("  ✅ Section 1: Dataset loading and sample exploration")
print("  ✅ Section 2: Standalone SQL evaluation")
print("  ✅ Section 3: Langfuse deployment verification")
print("  ✅ Section 4: Langfuse-integrated evaluation")
print("\n🚀 The interactive SQL evaluation notebook is ready for use!")
